In [131]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
    
import sys
sys.path.append("./")
from FastObiwan import * 
import fitsio

In [132]:
Obiwan_LRG = fitsio.read("/global/cfs/cdirs/desi/survey/catalogs/image_simulations/LRG/NGC/Obiwan_LRGs.fits", \
                         columns = ["brickname", "sim_ra", "sim_dec", "sim_gflux", "sim_rflux", "sim_zflux", "sim_w1", "mw_transmission_w1", "sim_rhalf", "sim_e1", "sim_e2","sim_sersic_n"])

#loop here
brickname = '2383p305'
Obiwan_LRG = Obiwan_LRG[Obiwan_LRG['brickname'] == brickname]

Obiwan_Fast = LegacysurveyCCDList(brickname)
Obiwan_Fast.read_sim_gal( Obiwan_LRG )
#loop here
Obiwan_Fast.get_source(0)
#loop here
L_CCDs = len( Obiwan_Fast.ccd_list)

tims = []
for i in range(L_CCDs):
    Obiwan_Fast.init_one_ccd(i)
    Obiwan_Fast.set_local()
    Obiwan_Fast.gaussian_background()
    Obiwan_Fast.resample_image()
    Obiwan_Fast.resample_psf()
    Obiwan_Fast.finalize_tim()
    tim_i = Obiwan_Fast.final_tim
    tims.append(tim_i)
    #print(Obiwan_Fast.gain, Obiwan_Fast.zpscale, Obiwan_Fast.sig1,Obiwan_Fast.nano2e)

In [133]:
Obiwan_Fast.get_source(0)
print("true flux")
print( Obiwan_Fast.source_i.brightness.g,Obiwan_Fast.source_i.brightness.r,Obiwan_Fast.source_i.brightness.z )

(0.65924746, 1.8840947, 4.1069384)

In [136]:
Obiwan_Fast.get_source(0)
Obiwan_Fast.source_i.brightness.g = Obiwan_Fast.source_i.brightness.g*(1+np.random.normal()*0.5)
Obiwan_Fast.source_i.brightness.r = Obiwan_Fast.source_i.brightness.r*(1+np.random.normal()*0.5)
Obiwan_Fast.source_i.brightness.z = Obiwan_Fast.source_i.brightness.z*(1+np.random.normal()*0.5)

In [137]:
print("guess flux")
Obiwan_Fast.source_i.brightness.g,Obiwan_Fast.source_i.brightness.r,Obiwan_Fast.source_i.brightness.z

guess flux


(0.998210298166907, 1.0151240707545597, 4.508360945647031)

In [138]:
new_tractor = Tractor( tims, [Obiwan_Fast.source_i])
new_tractor.freezeParam('images')

In [139]:
for i in range(50):
    dlnp,X,alpha = new_tractor.optimize()
    print('dlnp', dlnp)
    if dlnp < 1e-5:
        break

dlnp 472.88686669644085
dlnp 268.2909795548767
dlnp 42.0194631851482
dlnp 18.779871163147618
dlnp 4.460379011317855
dlnp 2.1998997103364673
dlnp 0.546522276577889
dlnp 0


In [140]:
b = Obiwan_Fast.source_i.brightness

In [143]:
print("fitted flux")
b.g, b.r, b.z

fitted flux


(0.8420217032889254, 1.5016950503413156, 4.363751307402345)

In [144]:
Obiwan_Fast.source_i

SersicGalaxy(pos=RaDecPos[238.43327189211578, 30.382086031371017], brightness=NanoMaggies: g=22.7, r=22.1, z=20.9, shape=re=0.690072, e1=-0.276785, e2=0.202344, sersicindex=0.500)

In [150]:
Obiwan_Fast.get_source(0)
Obiwan_Fast.source_i.brightness.g = Obiwan_Fast.source_i.brightness.g*(1+np.random.normal()*0.05)
Obiwan_Fast.source_i.brightness.r = Obiwan_Fast.source_i.brightness.r*(1+np.random.normal()*0.05)
Obiwan_Fast.source_i.brightness.z = Obiwan_Fast.source_i.brightness.z*(1+np.random.normal()*0.05)

print("guess flux")
Obiwan_Fast.source_i.brightness.g,Obiwan_Fast.source_i.brightness.r,Obiwan_Fast.source_i.brightness.z

guess flux


(0.6567202539904605, 2.106338229727914, 4.432322764550943)

In [151]:
new_tractor = Tractor( tims, [Obiwan_Fast.source_i])
new_tractor.freezeParam('images')

for i in range(50):
    dlnp,X,alpha = new_tractor.optimize()
    print('dlnp', dlnp)
    if dlnp < 1e-5:
        break

dlnp 38.93269123369828
dlnp 6.669643624292803
dlnp 1.066012730341754
dlnp 0.24081263107655104
dlnp 0.014743693041964434
dlnp 0


In [152]:
print("fitted flux")
b.g, b.r, b.z

fitted flux


(0.8420217032889254, 1.5016950503413156, 4.363751307402345)

In [ ]:
#The fitting result depends on intial guess? 